In [1]:
!pip install --upgrade pip
!pip install python-binance

# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

from datetime import timezone

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [2]:
target_period_hours = 20 * 24 + 23
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 7)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = 'https://www.deribit.com/api/v2/public'

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)

puts = trading_model.puts
calls = trading_model.calls
target_expiry = trading_model.target_expiry
put_strikes = trading_model.put_strikes
call_strikes = trading_model.call_strikes
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)

In [4]:
#
index_put = strikes_universe_size - 2
index_call = strikes_universe_size + 2
#
cost, value = (option_chain_df.iloc[index_put]['put_ask'] + option_chain_df.iloc[index_call]['call_ask'], 
 option_chain_df.iloc[index_put]['value_put_pct'] + option_chain_df.iloc[index_call]['value_call_pct']
)
print(f'target expiry: {target_expiry.astimezone(timezone.utc)} ({remaining_hours} hours left)')
print(f'current price: {underlying_price}')
print(f'trading put {option_chain_df.iloc[index_put].name:.0f} and call {option_chain_df.iloc[index_call].name:.0f}')
print(f'cost: {cost:.3f} / value: {value:.3f}, benefit/cost = {value / cost:.1f}x')
print(f'($) cost: {cost * underlying_price:.2f} / value: {value * underlying_price:.2f}, average gain = {(value - cost) * underlying_price:.2f}')

option_chain_df

target expiry: 2024-02-16 08:00:00+00:00 (502 hours left)
current price: 2208.0
trading put 2100 and call 2300
cost: 0.043 / value: 0.093, benefit/cost = 2.2x
($) cost: 94.94 / value: 205.42, average gain = 110.48


,value_call,value_put,value_call_median,value_put_median,value_call_pct,call_ask,value_put_pct,put_ask
strike,,,,,,,,
2000.0,300.472245,53.516773,206.035361,0.000000,NaN,0.1155,0.024238,0.0110
2050.0,262.916821,65.961349,156.035361,0.000000,NaN,0.0965,0.029874,0.0145
2100.0,227.735064,80.779592,106.035361,0.000000,NaN,0.0790,0.036585,0.0200
2150.0,195.870168,98.914695,56.035361,0.000000,NaN,0.0570,0.044798,0.0270
2200.0,168.410156,121.454683,6.035361,0.000000,NaN,0.0435,0.055007,0.0360
2250.0,144.883420,147.927947,0.000000,43.964639,0.065617,0.0315,NaN,0.0475
2300.0,124.643280,177.687807,0.000000,93.964639,0.056451,0.0230,NaN,0.0625
2350.0,106.696941,209.741469,0.000000,143.964639,0.048323,0.0165,NaN,15.0000
2400.0,90.856296,243.900824,0.000000,193.964639,0.041149,0.0120,NaN,15.0000
